<a href="https://colab.research.google.com/github/LxYuan0420/nlp/blob/main/notebooks/finetuning_distilbert_base_uncased_nli_triplet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install -q datasets sentence-transformers
#!pip install accelerate -U

In [1]:
from datasets import load_dataset
from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
    SentenceTransformerModelCardData,
)
from sentence_transformers.losses import MultipleNegativesRankingLoss
from sentence_transformers.training_args import BatchSamplers
from sentence_transformers.evaluation import TripletEvaluator

In [2]:
# 1. Load a model to finetune with 2. (Optional) model card data
model = SentenceTransformer(
    "distilbert/distilbert-base-uncased",
    model_card_data=SentenceTransformerModelCardData(
        language="en",
        license="apache-2.0",
        model_name="Distillbert base uncased model trained on AllNLI triplets",
    )
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
# 2. Load a dataset to finetune on
dataset = load_dataset("sentence-transformers/all-nli", "triplet")
train_dataset = dataset["train"].select(range(150000))
eval_dataset = dataset["dev"]
test_dataset = dataset["test"]

In [4]:
# 3. define a lose function
loss = MultipleNegativesRankingLoss(model)

In [5]:
# 4. Specifc training arguments
args = SentenceTransformerTrainingArguments(
    # Required parameter:
    output_dir="models/distilbert-base-uncased-nli-triplet",
    # Optional training parameters:
    num_train_epochs=1,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    warmup_ratio=0.1,
    fp16=True,  # Set to False if you get an error that your GPU can't run on FP16
    bf16=False,  # Set to True if you have a GPU that supports BF16
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # MultipleNegativesRankingLoss benefits from no duplicate samples in a batch
    # Optional tracking/debugging parameters:
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    logging_strategy="epoch",
    #run_name="distilbert-base-uncased-nli-triplet",  # Will be used in W&B if `wandb` is installed
)

In [6]:
#5 Create an evaluator % evaluate the base model
dev_evaluator = TripletEvaluator(
    anchors=eval_dataset["anchor"],
    positives=eval_dataset["positive"],
    negatives=eval_dataset["negative"],
    batch_size=16,
    name="distilbert-base-uncased-nli-triplet-dev",
)

dev_evaluator(model)

{'distilbert-base-uncased-nli-triplet-dev_cosine_accuracy': 0.712636695018226,
 'distilbert-base-uncased-nli-triplet-dev_dot_accuracy': 0.34234507897934385,
 'distilbert-base-uncased-nli-triplet-dev_manhattan_accuracy': 0.714003645200486,
 'distilbert-base-uncased-nli-triplet-dev_euclidean_accuracy': 0.7088396111786148,
 'distilbert-base-uncased-nli-triplet-dev_max_accuracy': 0.714003645200486}

In [7]:
# 6. Create a trainer & train
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss=loss,
    evaluator=dev_evaluator,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Distilbert-base-uncased-nli-triplet-dev Cosine Accuracy,Distilbert-base-uncased-nli-triplet-dev Dot Accuracy,Distilbert-base-uncased-nli-triplet-dev Manhattan Accuracy,Distilbert-base-uncased-nli-triplet-dev Euclidean Accuracy,Distilbert-base-uncased-nli-triplet-dev Max Accuracy
1,1.265000,0.643917,0.903098,0.110723,0.900213,0.900972,0.903098


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

TrainOutput(global_step=2344, training_loss=1.2650227725709258, metrics={'train_runtime': 2364.6432, 'train_samples_per_second': 63.435, 'train_steps_per_second': 0.991, 'total_flos': 0.0, 'train_loss': 1.2650227725709258, 'epoch': 1.0})

In [8]:
# 7. check the finetuned model on dev set
dev_evaluator(model)

{'distilbert-base-uncased-nli-triplet-dev_cosine_accuracy': 0.9030984204131227,
 'distilbert-base-uncased-nli-triplet-dev_dot_accuracy': 0.11072296476306197,
 'distilbert-base-uncased-nli-triplet-dev_manhattan_accuracy': 0.9002126366950183,
 'distilbert-base-uncased-nli-triplet-dev_euclidean_accuracy': 0.9009720534629405,
 'distilbert-base-uncased-nli-triplet-dev_max_accuracy': 0.9030984204131227}

In [9]:
# 8. check the trained model on test set
test_evaluator = TripletEvaluator(
    anchors=test_dataset["anchor"],
    positives=test_dataset["positive"],
    negatives=test_dataset["negative"],
    name="distilbert-base-uncased-nli-triplet-test",
)

test_evaluator(model)

{'distilbert-base-uncased-nli-triplet-test_cosine_accuracy': 0.9129974277500378,
 'distilbert-base-uncased-nli-triplet-test_dot_accuracy': 0.10379785141473748,
 'distilbert-base-uncased-nli-triplet-test_manhattan_accuracy': 0.911484339536995,
 'distilbert-base-uncased-nli-triplet-test_euclidean_accuracy': 0.9105764866091693,
 'distilbert-base-uncased-nli-triplet-test_max_accuracy': 0.9129974277500378}